In [1]:
from langchain_groq import ChatGroq
import os

In [2]:
import chromadb

In [3]:
GROG_API_KEY = os.environ.get("MY_GROG_API_KEY")

In [4]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    api_key=GROG_API_KEY,
    max_tokens=None,
    max_retries=2
)

In [5]:
#client = chromadb.Client()
#collection = client.create_collection(name="cold_mail")
#collection = client.create_collection(name="cold_email")

Capturing Website Data

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-43863")
page_data = loader.load().pop().page_content
print(page_data)

In [7]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """ 
    ### SRAPPED TEST FROM WEBSITE
    {page_data}
    ### INSTRUCTION:
    The scrapped text is from the career's page of a website.
    Your job is extract the job postings and return them in JSON format containing following keys: 'role','experience','skills' and 'description'
    Only return valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

In [ ]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
jason_parser = JsonOutputParser()
jason_res = jason_parser.parse(res.content)
jason_res

Importing Dataset and Inserting into ChromaDB

In [ ]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

In [11]:
import uuid
client =chromadb.PersistentClient('portfoliostore')
collection = client.get_or_create_collection('portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(
            documents=row['Techstack'],
            metadatas={'links' : row['Links']},
            ids=[str(uuid.uuid4())]
        ) 


In [14]:
links = collection.query(query_texts=jason_res['skills'],n_results=2).get('metadatas')

In [13]:
prompt_email = PromptTemplate.from_template(
    """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)

In [18]:
job = jason_res
print(job)

{'role': 'Software Engineer III, ITC', 'experience': '5-8 years', 'skills': ['Marketing technology experience', 'Adobe toolset (AEP, AJO, CJA)', 'Python or Java', 'Sql', 'Data Engineering', 'Airflow, EMR, Hive, Spark', 'Microservices architecture', 'Domain driven design', 'RESTful API’s', 'CI/CD', 'DevOps', 'Data modeling', 'ETL development', 'Data streaming'], 'description': 'We are looking for a Software engineer, to be a part of Nike’s Marketing Technology organization. This position will play a critical role, in developing Nike’s Customer Data platform, which supports and promotes our products and provides a unique user experience to all our customers in 166 countries globally. We are looking for someone to have a strong Marketing Automations skills and analytical background with a focus of delivering an omnichannel consumer experience through data driven marketing.'}


In [ ]:
chain_email = prompt_email | llm
res = chain_email.invoke({'job_description': str(job),'link_list': links})
print(res.content)